# Beispiel-Implementierung: Lokale Suchmaschine

## Ziel der Beispiel-Implementierung
Im Folgenden wird eine Anwendung der zuvor theoretisch diskutierten Inhalte liefern. Dabei soll eine lokale Suchmaschine entwickelt werden, welche in der Lage ist, pdf-Dateien auf einem lokalen Computer-System zu parsen, in einen invertierten Index aufzunehmen sowie Suchanfragen eines Benutzers sinnvoll zu beantworten. <br>
Zur Relevanz-Bestimmung der Dokumente wird das TF-IDF-Maß, welches bereits vorgestellt wurde, genutzt. Um den Index zu speichern, wird die von Python mitgelieferte Datenstruktur "dictionary", welche im Grunde eine Hashmap ist, genutzt.
Weiter werden einige Bibliotheken eingesetzt, welche einige Vorarbeit leisten und damit den Code der Beispiel-Implementierung auf das Wesentliche beschränken. So soll die grundlegende Arbeitsweise eines Information Retrieval-Systems dargelegt werden.

## Genutzte Bibliotheken
Bevor mit der eigentlichen Implementierung der lokalen Suchmaschine begonnen werden kann, müssen einige Bibliotheken eingebunden werden. Darunter fallen Apache Tika, das Math-Modul von Python, os (um auf die Directories zugreifen zu können) (und noch weitere, bei Bedarf einfügen!). <br>
Tika liefert eine Parser, mit dessen Hilfe der Text aus - unter anderem - pdf-Dateien extrahiert werden kann.
Mit dem Aufruf _parser.from_\__file(file)_ kann eine pdf-Datei in reinen Text umgewandelt werden. Die Funktion liefert ein Dictionary zurück, welches einen Key _content_ besitzt, über den auf den Inhalt der pdf-Datei zugegriffen werden kann.


In [1]:
from tika import parser
import math
import os

## Vorarbeit: pdf-Dateien verarbeiten
Um den invertierten Index aufbauen zu können, muss der Text aus den pdf-Dateien extrahiert werden. Anschließend kann die Tokenerzeigung durchgeführt werden, darauf aufbauend werden die Terme erzeugt, die dann im Index aufgenommen werden. Im gleichen Schritt werden die Informationen gesammelt, die notwendig sind, um mittels TF-IDF die Relevanz eines Dokuments zu bestimmen. <br>
Diese Schritte werden im Folgenden anhand des Codes diskutiert.
<br>


In [ ]:
def countDocuments():
    for subdir, dir, files in os.walk("C:/"):
        #prüfe, ob pdf in files
        #wenn ja, Counter für jedes pdf um 1 hochsetzen, sonst weiter